In [22]:
# Import Librairies
# !pip install nltk
import pandas as pd
import re
import sys
import json
!pip install gspread_dataframe gspread oauth2client
!pip install gspread-pandas
from gspread_pandas import Spread, Client
from google.oauth2.service_account import Credentials

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
!{sys.executable} -m pip install pygsheets
import pygsheets
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.cluster import DBSCAN
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer
from collections import Counter

import csv
# from google.colab import files
# uploaded = files.upload()

In [23]:
# Import our Google Search Console CSV Data into a Dataframe
df = pd.read_csv('Queries.csv', delimiter=',')
df.head(20)


,Top queries,Clicks,Impressions,CTR,Position
0,adelphi living,289,834,34.65%,1.07
1,adelphi,15,298,5.03%,9.97
2,sil providers perth,12,90,13.33%,3.26
3,supported independent living perth,7,323,2.17%,10.77
4,adelphi living mandurah,7,30,23.33%,1.03
5,disability accommodation perth,3,641,0.47%,31.52
6,ndis providers near me,3,45,6.67%,1.00
7,sda providers perth,3,23,13.04%,7.48
8,disability housing perth,2,541,0.37%,31.37
9,ndis providers perth,2,258,0.78%,52.90


In [24]:
# Define a variable to hold our 'Top queries' column only
top_queries = df['Top queries']

# Create a list of keywords and populate it from our dataframe
keywords = []

for x in top_queries:
  keywords.append(x)

# Display keywords list
# keywords

In [25]:
# Creating Class Object and assign english as language parameter
snow_stemmer = SnowballStemmer(language='english')

# Stem every keyword from our keywords list and append them into a new list
stem_words = []
for keyword in keywords:
    x = snow_stemmer.stem(keyword)
    stem_words.append(x)

keywords_vs_stem = pd.DataFrame({'Keywords': keywords,
                                'Stem Words': stem_words})
keywords_vs_stem

,Keywords,Stem Words
0,adelphi living,adelphi liv
1,adelphi,adelphi
2,sil providers perth,sil providers perth
3,supported independent living perth,supported independent living perth
4,adelphi living mandurah,adelphi living mandurah
...,...,...
713,disabled living manchester,disabled living manchest
714,disability insurance perth,disability insurance perth
715,specialist disability housing,specialist disability h
716,disability support providers,disability support provid


In [26]:
# Keyword Clustering with sklearn and TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.2, max_features=10000,min_df=0.01, stop_words='english',use_idf=True, ngram_range=(1,2))
tfidf_matrix = tfidf_vectorizer.fit_transform(keywords)
ds = DBSCAN(eps=0.7, min_samples=5).fit(tfidf_matrix)
clusters = ds.labels_

In [27]:
# Create a dataframe for our clusters
cluster_df = pd.DataFrame(clusters, columns=['Cluster Number'])

# Merge our cluster df with our top queries df
result = pd.merge(cluster_df, top_queries, left_index=True, right_index=True)

# We group our keywords by cluster number and separate them with a comma
clusters_queries = result.groupby(['Cluster Number'])['Top queries'].apply(', '.join).reset_index()
# clusters_queries.to_csv("clusters.csv",index=False)

# Extend Column and Row width
pd.set_option('max_colwidth', 400)
pd.set_option('max_rows', 100)

clusters_queries


,Cluster Number,Top queries
0,-1,"sil providers perth, ndis providers near me, disability service providers perth, sil housing, adelphi house assisted living, adelphi housing, ndis support provider, sda accommodation, sil providers, independent living homes, sda perth, specialist support coordination perth, independent living near me, independent living solutions, independent living facilities near me, sda properties perth, as..."
1,0,"adelphi living, adelphi, adelphi living jobs, the adelphi, freedom adelphi, adelphi hotel perth, adelphi perth, adelphi beauty, logo adelphi, adelphi group, adelphi insurance, adelphi menu, adelphi freedom, adelphi logo, 27 adelphi road claremont, adelphi springs, freedom adelphi table, air adelphi, adelphi consult, adelphi echuca, adelphi portal, adelphi ceo, clicks adelphi, adelphi hotels, a..."
2,1,"supported independent living perth, supported independent living ndis, ndis supported independent living, supported independent living, supported independent living providers, independent living perth, independent living perth wa, supported independent living disability, living in perth, retirement living perth, supported independent living wa, sil supported independent living, community livin..."
3,2,"adelphi living mandurah, ndis mandurah, outdoor living mandurah, access housing mandurah, living in mandurah, mandurah accommodation, accommodation in mandurah perth, accomodation mandurah, accommodation in mandurah wa"
4,3,"sda providers perth, ndis providers perth, sda providers victoria, sda providers, it providers perth"
5,4,"disability housing perth, ndis housing perth, disability housing, sda housing perth, disability rental housing perth, specialist disability housing perth, disability housing solutions perth, disability housing options perth, physical disability housing perth, sda housing ndis, sda housing availability, sil disability housing, disability housing wa, independent living disability housing, sda ho..."
6,5,"sil ndis, sil, sil accommodation, sil and sda, sil accomodation, sda and sil, sil sda, sil assessment ndis, sil eligibility, sil perth, what is sil, sil supports, what is sil ndis, sil logo, sil living, sda and sil ndis, ndia sil, sil ltd, sda sil, sil ndia, what does sil include, difference between sil and sda, sil disability, sil insurance, sil., sil sda ndis, what is an sil ndis, who is sil..."
7,6,"disability supported accommodation perth, supported accommodation perth, supported accommodation ndis, supported accommodation for people with disabilities, ndis supported accommodation, supported accommodation, mental health supported accommodation perth, disability supported accommodation, shared supported accommodation disability services, shared supported accommodation"
8,7,"ndis sda, ndis sda application form, ndis sda properties perth, ndis sda rules, ndis sda eligibility, ndis sda funding, ndis sda fact sheet"
9,8,"ndis events perth, ndis perth, ndis mental health perth, ndis properties perth, perth living, ndis renovations perth, perth exhibitions, perth serviced apartment deal, connection perth, exhibitions perth, ndia perth, 6107 perth, exhibitions in perth, simple approach perth"


In [28]:
# Create an empty list to store our cluster names
cluster_name = []

# Use Iterrows and tfIdVectorizer to get all words and their TF-IDF Score per cluster
for index, row in clusters_queries.iterrows():
    
    queries = row['Top queries']
    queries = [queries]

    # Add a stop_words function to drop irrelevant words (ex: “a”, “the”, “is”, “are”, "and" etc...)
    my_stop_words = text.ENGLISH_STOP_WORDS

    # Run tfIdfVectorizer for each cluster
    tfIdfVectorizer=TfidfVectorizer(use_idf=True, stop_words=my_stop_words)
    tfIdf = tfIdfVectorizer.fit_transform(queries)
    query_score_cluster = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF Score"])
    query_score_cluster = query_score_cluster.sort_values('TF-IDF Score', ascending=False)

    # Get the top 3 queries (selected by highest scores) per cluster and get them into lists
    top_three_queries = query_score_cluster.index[:3]
    top_three_queries_list = top_three_queries.values.tolist()
    top_three_queries_list = str(top_three_queries_list).replace(',', '')
    top_three_queries_list = str(top_three_queries_list).replace('[','').replace(']','')

    # Append our top 3 queries to our cluster_name list
    cluster_name.append(top_three_queries_list)
    
    # Create a Cluster Name dataframe    
    cluster_name_df = pd.DataFrame({'Cluster Name': cluster_name})

cluster_name_df.head(10)


,Cluster Name
0,'living' 'perth' 'ndis'
1,'adelphi' 'freedom' 'perth'
2,'living' 'independent' 'supported'
3,'mandurah' 'accommodation' 'living'
4,'providers' 'perth' 'sda'
5,'housing' 'disability' 'perth'
6,'sil' 'sda' 'ndis'
7,'accommodation' 'supported' 'disability'
8,'ndis' 'sda' 'application'
9,'perth' 'ndis' 'exhibitions'


In [29]:
# Merge cluster_name_df with clusters_queries so we can have them matching and double check the cluster names match with our queries
clusters = pd.merge(cluster_name_df, clusters_queries, left_index=True, right_index=True)
clusters

,Cluster Name,Cluster Number,Top queries
0,'living' 'perth' 'ndis',-1,"sil providers perth, ndis providers near me, disability service providers perth, sil housing, adelphi house assisted living, adelphi housing, ndis support provider, sda accommodation, sil providers, independent living homes, sda perth, specialist support coordination perth, independent living near me, independent living solutions, independent living facilities near me, sda properties perth, as..."
1,'adelphi' 'freedom' 'perth',0,"adelphi living, adelphi, adelphi living jobs, the adelphi, freedom adelphi, adelphi hotel perth, adelphi perth, adelphi beauty, logo adelphi, adelphi group, adelphi insurance, adelphi menu, adelphi freedom, adelphi logo, 27 adelphi road claremont, adelphi springs, freedom adelphi table, air adelphi, adelphi consult, adelphi echuca, adelphi portal, adelphi ceo, clicks adelphi, adelphi hotels, a..."
2,'living' 'independent' 'supported',1,"supported independent living perth, supported independent living ndis, ndis supported independent living, supported independent living, supported independent living providers, independent living perth, independent living perth wa, supported independent living disability, living in perth, retirement living perth, supported independent living wa, sil supported independent living, community livin..."
3,'mandurah' 'accommodation' 'living',2,"adelphi living mandurah, ndis mandurah, outdoor living mandurah, access housing mandurah, living in mandurah, mandurah accommodation, accommodation in mandurah perth, accomodation mandurah, accommodation in mandurah wa"
4,'providers' 'perth' 'sda',3,"sda providers perth, ndis providers perth, sda providers victoria, sda providers, it providers perth"
5,'housing' 'disability' 'perth',4,"disability housing perth, ndis housing perth, disability housing, sda housing perth, disability rental housing perth, specialist disability housing perth, disability housing solutions perth, disability housing options perth, physical disability housing perth, sda housing ndis, sda housing availability, sil disability housing, disability housing wa, independent living disability housing, sda ho..."
6,'sil' 'sda' 'ndis',5,"sil ndis, sil, sil accommodation, sil and sda, sil accomodation, sda and sil, sil sda, sil assessment ndis, sil eligibility, sil perth, what is sil, sil supports, what is sil ndis, sil logo, sil living, sda and sil ndis, ndia sil, sil ltd, sda sil, sil ndia, what does sil include, difference between sil and sda, sil disability, sil insurance, sil., sil sda ndis, what is an sil ndis, who is sil..."
7,'accommodation' 'supported' 'disability',6,"disability supported accommodation perth, supported accommodation perth, supported accommodation ndis, supported accommodation for people with disabilities, ndis supported accommodation, supported accommodation, mental health supported accommodation perth, disability supported accommodation, shared supported accommodation disability services, shared supported accommodation"
8,'ndis' 'sda' 'application',7,"ndis sda, ndis sda application form, ndis sda properties perth, ndis sda rules, ndis sda eligibility, ndis sda funding, ndis sda fact sheet"
9,'perth' 'ndis' 'exhibitions',8,"ndis events perth, ndis perth, ndis mental health perth, ndis properties perth, perth living, ndis renovations perth, perth exhibitions, perth serviced apartment deal, connection perth, exhibitions perth, ndia perth, 6107 perth, exhibitions in perth, simple approach perth"


In [30]:
splited_clusters = pd.concat([pd.Series(row['Cluster Name'], row['Top queries'].split(','))              
                    for _, row in clusters.iterrows()]).reset_index()

In [31]:
# raw_happiness_df.rename(columns={'Country name': 'Country', 
splited_clusters = splited_clusters.rename(columns={'index': 'Top queries', 0: 'Cluster Name'})
# splited_clusters.insert(1, "Top queries", "")
splited_clusters

,Top queries,Cluster Name
0,sil providers perth,'living' 'perth' 'ndis'
1,ndis providers near me,'living' 'perth' 'ndis'
2,disability service providers perth,'living' 'perth' 'ndis'
3,sil housing,'living' 'perth' 'ndis'
4,adelphi house assisted living,'living' 'perth' 'ndis'
...,...,...
717,accommodation in rockingham wa,'rockingham' 'accommodation' 'wa'
718,rockingham accommodation wa,'rockingham' 'accommodation' 'wa'
719,rockingham wa accommodation,'rockingham' 'accommodation' 'wa'
720,rockingham accommodation,'rockingham' 'accommodation' 'wa'


In [32]:
splited_clusters.insert(2, "Clicks", "")
splited_clusters.insert(3, "Impressions", "")
splited_clusters.insert(4, "CTR", "")
splited_clusters.insert(5, "Position", "")

In [33]:
splited_clusters

,Top queries,Cluster Name,Clicks,Impressions,CTR,Position
0,sil providers perth,'living' 'perth' 'ndis',,,,
1,ndis providers near me,'living' 'perth' 'ndis',,,,
2,disability service providers perth,'living' 'perth' 'ndis',,,,
3,sil housing,'living' 'perth' 'ndis',,,,
4,adelphi house assisted living,'living' 'perth' 'ndis',,,,
...,...,...,...,...,...,...
717,accommodation in rockingham wa,'rockingham' 'accommodation' 'wa',,,,
718,rockingham accommodation wa,'rockingham' 'accommodation' 'wa',,,,
719,rockingham wa accommodation,'rockingham' 'accommodation' 'wa',,,,
720,rockingham accommodation,'rockingham' 'accommodation' 'wa',,,,


In [50]:
x = df['Top queries'].tolist()
y = 

['sil providers perth',
 ' ndis providers near me',
 ' disability service providers perth',
 ' sil housing',
 ' adelphi house assisted living',
 ' adelphi housing',
 ' ndis support provider',
 ' sda accommodation',
 ' sil providers',
 ' independent living homes',
 ' sda perth',
 ' specialist support coordination perth',
 ' independent living near me',
 ' independent living solutions',
 ' independent living facilities near me',
 ' sda properties perth',
 ' assisted living apartments for disabled',
 ' mental health housing wa',
 ' atsa',
 ' disabled persons housing service',
 ' respite care perth',
 ' supported independant living facility',
 ' disability solutions for independent living',
 ' sil services',
 ' sda services',
 ' ndis services perth',
 ' home for disabled',
 ' what are the ndis supported indepented living',
 ' sda disability',
 ' respite accommodation ndis',
 ' disabled living accommodation',
 ' respite care perth wa',
 ' perth disability expo 2021',
 ' adelphi care',
 ' at

In [43]:
clicks_list = []

for index, row in splited_clusters.iterrows():
#     x = df['Top queries']
#     for word in keywords:
    if x in row['Top queries']:
        clicks_list.append(df['Clicks'])
#     break
clicks_list

TypeError: 'in <string>' requires string as left operand, not Series

In [35]:
clicks_list_bis = []

for index, row in df.iterrows():
#     for word in keywords:
    if word in row['Top queries']:
        clicks_list_bis.append(df['Clicks'])
#     break
clicks_list

[]

In [38]:
click_list = []
for index, row in splited_clusters.iterrows():
    
    query = row['Top queries']
    if query in df['Top queries']:
        row_index = df.index[df['Top queries'] == query][0]
        clicks = df.iloc[row_index, 1]
        splited_clusters.iloc[index, 3] = clicks
        click_list.append(df['Top queries'])
        

In [39]:
splited_clusters

,Top queries,Cluster Name,Clicks,Impressions,CTR,Position
0,sil providers perth,'living' 'perth' 'ndis',,,,
1,ndis providers near me,'living' 'perth' 'ndis',,,,
2,disability service providers perth,'living' 'perth' 'ndis',,,,
3,sil housing,'living' 'perth' 'ndis',,,,
4,adelphi house assisted living,'living' 'perth' 'ndis',,,,
...,...,...,...,...,...,...
717,accommodation in rockingham wa,'rockingham' 'accommodation' 'wa',,,,
718,rockingham accommodation wa,'rockingham' 'accommodation' 'wa',,,,
719,rockingham wa accommodation,'rockingham' 'accommodation' 'wa',,,,
720,rockingham accommodation,'rockingham' 'accommodation' 'wa',,,,


In [ ]:
final_df = pd.merge(splited_clusters, df, how='inner', on='Top queries', right_index=True)
final_df.reset_index
final_df

In [ ]:
# Add empty cluster columns to our very first dataframe
# df.insert(1, "Cluster Number", "")


In [ ]:
# df.insert(2, "Cluster Name", "")

In [ ]:
test = []

for index, row in clusters.iterrows():
    for word in keywords:
        if word in row['Top queries']:
            test.append(clusters['Cluster Name'])
#     break
test

In [ ]:
# test = []

# for index, row in df.iterrows():
#     for word in keywords:
#         if word in clusters['Top queries']:
#             test.append(clusters['Cluster Name'])
# test

In [ ]:
for index, row in clusters.iterrows():
  for word in keywords:
    if word in row['Top queries']:
        df.loc[df[index, 'Top queries']] = clusters.loc[clusters['Top queries']]
#     else:
#         df.loc[df['Top queries']] = df.loc[df['Top queries']]

In [ ]:
test_2 = []

# df_final = df.set_index("Top queries")
# clusters_final = clusters.set_index("Top queries")


# for index, row in clusters_final.iterrows():
#   for word in keywords:
#     if word in row['Top queries']:
#         df_final.loc[df_final['Top queries']] == clusters_final.loc[clusters_final['Top queries']]
#     else:
#         df_final.loc[df['Top queries']] == df_final.loc[df['Top queries']]
#         df['Cluster Name'].append(clusters['Cluster Number'])
# df

In [ ]:
# clusters.loc[clusters['Top queries'].isin(df['Top queries']),['Cluster Number','Cluster Name']] = df[['Cluster Number', 'Cluster Name']].values

In [ ]:
df

In [ ]:
# df.to_csv('df_csv.csv', index=False)

In [ ]:
# scope = ['https://www.googleapis.com/auth/spreadsheets',
#         'https://www.googleapis.com/auth/drive']

In [ ]:
# creds = Credentials.from_service_account_file("default.json", scopes=scope)

In [ ]:
# client = gspread.authorize(creds)

In [ ]:
# google_sh = client.open("keyword_clustering")

In [ ]:
# sheet1 = google_sh.get_worksheet(0)

In [ ]:
# df_test = set_with_dataframe(sheet1, df)

In [ ]:
# df_test_read = get_as_dataframe(sheet1)
# df_test_read.head(20)